In [ ]:
# Third-party
import numpy as np

Make some fake Gaia uncertainties with the same column names as the simulated data:

In [ ]:
A = np.random.uniform(size=(32,5,5))
cov = 0.5 * np.einsum('nij,nkj->nik', A, A)

In [ ]:
data = dict()
for i,name1 in enumerate(['ra', 'dec', 'parallax', 'pmra', 'pmdec']):
    data['{}_error'.format(name1)] = np.sqrt(cov[:,i,i])
    for j,name2 in enumerate(['ra', 'dec', 'parallax', 'pmra', 'pmdec']):
        if j >= i: continue
        data['{}_{}_corr'.format(name1,name2)] = cov[:,i,j] / (np.sqrt(cov[:,i,i]*cov[:,j,j]))

In [ ]:
def construct_cov(gaia_data):
    """
    If the real data look like the simulated data, Gaia will provide 
    correlation coefficients and standard deviations for 
    (ra,dec,parallax,pm_ra,pm_dec), but we probably want to turn that
    into a covariance matrix.
    """
    names = ['ra', 'dec', 'parallax', 'pmra', 'pmdec']
    
    n = len(gaia_data['ra_error'])
    C = np.zeros((n,len(names),len(names)))
    
    # pre-load the diagonal
    for i,name in enumerate(names):
        full_name = "{}_error".format(name)
        C[:,i,i] = gaia_data[full_name]**2
    
    for i,name1 in enumerate(names):
        for j,name2 in enumerate(names):
            if j >= i: continue    
            full_name = "{}_{}_corr".format(name1, name2)
            C[...,i,j] = gaia_data[full_name]*np.sqrt(C[...,i,i]*C[...,j,j])
            C[...,j,i] = gaia_data[full_name]*np.sqrt(C[...,i,i]*C[...,j,j])
    
    return C    

In [ ]:
out_cov = construct_cov(data)

In [ ]:
assert np.allclose(out_cov, cov)